In [34]:
import pandas as pd
import pathlib
import numpy as np

# ------------------------------------------------------------------
# 1. Locate and load every “Deep *.xlsx” file -----------------------
# ------------------------------------------------------------------
DATA_DIR = pathlib.Path(r"C:\Users\45237\OneDrive - Danmarks Tekniske Universitet\Skrivebord\Statistical analysis ML models\Skemaer")        # adjust filepath
files = sorted(DATA_DIR.glob("Deep *.xlsx"))  # Deep 1.xlsx … Deep 36.xlsx

if not files:
    raise FileNotFoundError("No files matching 'Deep *.xlsx' were found.")

runs = []
for f in files:
    df = pd.read_excel(f)
    df["run_id"] = f.stem          # e.g. “Deep 23”
    runs.append(df)

raw = pd.concat(runs, ignore_index=True)

# ------------------------------------------------------------------
# 2. Identify which option was chosen in each row ------------------
# ------------------------------------------------------------------
response_cols = ["Strongly disagree", "Disagree", "Agree", "Strongly agree"]

long = (
    raw
    .melt(id_vars=["Questions:", "Axis", "Direction", "run_id"],
          value_vars=response_cols,
          var_name="response",
          value_name="selected")
    .query("selected == 1")
    .drop(columns="selected")
)

# ------------------------------------------------------------------
# 3. Map responses to numeric scores, applying direction (+ / –) ----
# ------------------------------------------------------------------
score_map = {
    "Strongly disagree": -2,
    "Disagree":          -1,
    "Agree":              1,
    "Strongly agree":     2,
}



long["signed_score"] = long["response"].map(score_map) * (
    long["Direction"].apply(lambda d: -1 if d.strip() == "-" else 1)
)

# ------------------------------------------------------------------
# 4. Measure variability *by axis* and rank the top 5 --------------
# ------------------------------------------------------------------
stats = (
    long
    .groupby(["Axis", "Questions:"], as_index=False)
    .agg(
        n_runs     = ("signed_score", "size"),
        std_dev    = ("signed_score", "std"),
        iqr        = ("signed_score", lambda s: s.quantile(0.75) - s.quantile(0.25)),
        resp_mode  = ("response",    lambda s: s.mode().iat[0])
    )
    .sort_values(["Axis", "std_dev"], ascending=[True, False])
)

# Split and pick the five most variable questions on each axis
top_E = stats.query("Axis == 'E'").head(3)
top_S = stats.query("Axis == 'S'").head(9)

# ------------------------------------------------------------------
# 5. Display neatly -------------------------------------------------
# ------------------------------------------------------------------
pd.set_option("display.max_colwidth", None)
print("Top-5 most variable ECONOMIC statements:")
print(top_E[["Questions:", "std_dev", "iqr", "resp_mode"]], end="\n\n")
print('Explained Variance of top 3 Economic questions:')
print(0.878310**2/sum(stats.query("Axis == 'E'")['std_dev']**2))
print(0.487136**2/sum(stats.query("Axis == 'E'")['std_dev']**2))
print(0.467177**2/sum(stats.query("Axis == 'E'")['std_dev']**2))
print(sum(top_E["std_dev"]**2)/sum(stats.query("Axis == 'E'")['std_dev']**2))

print("Top-9 most variable SOCIAL statements:")
print(top_S[["Questions:", "std_dev", "iqr", "resp_mode"]])

print('Explained Variance of top 9 Social questions:')
print(sum(top_S["std_dev"]**2)/sum(stats.query("Axis == 'S'")['std_dev']**2))



Top-5 most variable ECONOMIC statements:
                                                                              Questions:  \
8                                    Land shouldn’t be a commodity to be bought and sold   
17  Those with the ability to pay should have access to higher standards of medical care   
4                         Governments should penalise businesses that mislead the public   

     std_dev  iqr          resp_mode  
8   0.878310  0.5              Agree  
17  0.487136  1.0  Strongly disagree  
4   0.467177  1.0              Agree  

Explained Variance of top 3 Economic questions:
0.5107723881692067
0.1571202669180032
0.14450892283475567
0.8124014713610088
Top-9 most variable SOCIAL statements:
                                                                                                 Questions:  \
38                                   Making peace with the establishment is an important aspect of maturity   
22                              Abortion, when 